# Autoencoder Model

## Training the model

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy import argmax

In [0]:
mnist = tf.keras.datasets.mnist

In [0]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
x_train, x_test = x_train.reshape(-1, 28 * 28), x_test.reshape(-1, 28 * 28)
singleton_shape = (1, 28 * 28)
encoding_dim = 8

encoder = tf.keras.models.Sequential([
  tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(28 * 28,)),
  tf.keras.layers.Dense(encoding_dim, activation=tf.nn.sigmoid),
])

decoder = tf.keras.models.Sequential([
  tf.keras.layers.Dense(encoding_dim, activation=tf.nn.sigmoid, input_shape=(encoding_dim,)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(28 * 28, activation=tf.nn.sigmoid),
])

In [0]:
autoencoder = tf.keras.models.Sequential([
  encoder,
  decoder
])

autoencoder.compile(optimizer='adam',
              loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [0]:
for i in range(5):
  fig = plt.figure()
  ax = fig.add_subplot(1,2,1)
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  plt.suptitle('encoded value: {}'.format(encoder.predict(
      x_test[i].reshape(singleton_shape))[0]))
  plt.imshow(autoencoder.predict(
    x_test[i].reshape(singleton_shape))[0].reshape((28, 28)),
    cmap='gray')
  ax = fig.add_subplot(1,2,2)
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  plt.imshow(x_test[i].reshape((28, 28)), cmap='gray')

## Saving Weights
These lines save our model weights

In [0]:
encoder.save('mnist_encoder.h5')
decoder.save('mnist_decoder.h5')

## Installing tensorflowjs_converter Utility
First, TensorFlow.js needs it's own virtual environment to operate, so we'll make one using python3-venv, but any utility virtual environment utility will do.

Note: jupyter notebooks are *NOT* made for this. You should run any shell commands you see here (lines starting with an exclamation mark) on the command line, specifically the ones to do with virtual environments. I am using notebooks for simplicity and illustrative purposes. The worst thing that can happen is that pip replaces the version of a bunch of packages, rendering your python packages unhappy. The reason it doesn't really matter on colab is that it is the code is running on an ephemeral machine which resets when you are done with it. Do *NOT* use notebooks for this on your local machine, you have been warned!

In [0]:
!apt-get install python3-venv

We create an virtual environment that we'll call "tensorflowjs-env"

In [0]:
!python3 -m venv .tensorflowjs-env

To install tensorflowjs, we first activate the environment. Then, we can safely install tensorflowjs. Once it's done, we deactivate the virtual environment.

In [0]:
!source .tensorflowjs-env/bin/activate
!pip3 install tensorflowjs
!deactivate

## Converting the Models into TensorFlow.js's Format
Now we create two directories, one for the encoder, and one for the decoder.
We then use the tensorflowjs_converter utility to convert the .h5 files into TensorFlow.js's format.

In [0]:
!source .tensorflowjs-env/bin/activate
!mkdir mnist_encoder
!mkdir mnist_decoder
!tensorflowjs_converter --input_format keras mnist_encoder.h5 mnist_encoder
!tensorflowjs_converter --input_format keras mnist_decoder.h5 mnist_decoder
!deactivate

Since this is colab, we'll zip and download the files.
On your local machine, you would just move the folders into the website directory.

In [0]:
!zip -r mnist_encoder.zip mnist_encoder
!zip -r mnist_decoder.zip mnist_decoder

In [0]:
from google.colab import files
files.download('mnist_encoder.zip')
files.download('mnist_decoder.zip')

## Importing the Model into the Browser
Workshop will continue with client-side code which can be found at https://github.com/symmetries/f2019workshop2 